In [28]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import random
from decouple import config

## Getting random User agents

In [11]:
with open('user_agents.txt','r') as f:
    user_agents = f.readlines()
user_agent = str(random.choice(user_agents)).strip()


## Getting links of each product

In [12]:
keyword = input('enter the keyword')
page = int(input('enter the number of pages you want to scrape'))
links = []
for page_no in range(1,int(page)):
    headers = {'user-agent':user_agent}
    url =f"https://www.amazon.in/s?k={keyword}&page={page_no}&qid=1638731354&ref=sr_pg_{page_no}"
    response = requests.get(url,headers=headers).content
    soup = bs(response,'lxml')
    all_product_soup = soup.findAll('a',attrs={"class":"a-link-normal a-text-normal"})
    for x in all_product_soup:
        links.append(x.attrs['href'] )

enter the keywordlaptops
enter the number of pages you want to scrape3


In [13]:
len(links)

35

## Getting Product Details

In [14]:
#use of headless selenium to get the dynamic content from amazon product page
data =[]
for product_link in links:
    path = config('PATH')
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(path,options=options)
    driver.get("https://www.amazon.in"+product_link)
    time.sleep(1)
    source_code = driver.page_source
    driver.quit()
    product_soup =  bs(source_code,'lxml')
    product_details={}
    for details in product_soup.find('table',attrs={'id':'productDetails_techSpec_section_1'}).tbody.findAll('tr'):
        product_details[details.th.text.rstrip()]=details.td.text.lstrip()

    data.append(product_details)
    
    

C:\Users\Anchal\AppData\Local\Temp/ipykernel_68056/1554890782.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path,options=options)


In [15]:
df1=pd.DataFrame(data)

In [16]:
df1.tail()

,Brand,Manufacturer,Series,Colour,Form Factor,Item Height,Item Width,Standing screen display size,Resolution,Product Dimensions,...,Screen Resolution,Voltage,Front Webcam Resolution,Hard Disk Rotational Speed,Aspect Ratio,Microphone Form Factor,Microphone Technology,Processor Socket,Number of Lithium Metal Cells,Battery Charge Cycles
30,‎MSI,‎MSI COMPUTER,‎GF75 Thin 10SC-095IN,‎Black,‎Laptop,‎22 Millimeters,‎26 Centimeters,‎44 Centimetres,‎1920 x 1080 Pixels,‎39.7 x 26 x 2.2 cm; 2.2 Kilograms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,‎AVITA,‎Alco Holdings Limited,‎AVITA Essential,‎Gray,‎Laptop,‎20 Millimeters,‎32.1 Centimeters,‎14 Inches,‎1920 x 1080,‎22.5 x 32.1 x 2 cm; 1.37 Kilograms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,‎Lenovo,‎Lenovo,‎IdeaPad 1 11IGL05,‎Platinum Grey,‎Thin & Light,‎17 Millimeters,‎20 Centimeters,‎11.6 Inches,‎1366 x 768 Pixels,‎28.8 x 20 x 1.7 cm; 1.2 Kilograms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,‎Dell,‎Dell India Pvt Ltd,‎VOSTRO 3401,‎Black,‎Laptop,‎20 Millimeters,‎24 Centimeters,‎14,‎1920x1080,‎32.8 x 24 x 2 cm; 1.64 Kilograms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,‎10 Hours
34,‎Dell,‎Dell Computers,NaN,‎Grey,‎Notebook,‎2 Inches,‎24 Inches,‎14 Inches,‎HD (1366 X 768) Pixels,‎83.82 x 60.96 x 5.08 cm; 1.75 Kilograms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df1.isnull().sum()

 Brand                             0
 Manufacturer                      0
 Series                           10
 Colour                           10
 Form Factor                       0
                                  ..
 Microphone Form Factor           34
 Microphone Technology            34
 Processor Socket                 34
 Number of Lithium Metal Cells    34
 Battery Charge Cycles            34
Length: 93, dtype: int64

In [22]:
df1.to_csv('amazon_data.csv',index=False)